# Опыт классификации комментариев товаров интернет-магазина с помощью классических моделей машинного обучения.
Целью проекта являлся подбор модели машинного обучения, которая бы разделяла комментарии от покупателей интернет-магазина на токсичные и нейтральные с качеством работы не менее 0.75 по метрике f1.

В ходе работы над проектом имеющиеся данные были изучены и предобработаны определенным образом с целью получить максимальный уровень качества  на одной из классических моделей машинного обучения и векторизации на основе метрики TF-IDF.
Исследование проводилось на основе  набора из 159292  размеченных текстов сообщений на английском языке.

В ходе изучения данных было выяснено, что в половине всех случаев токсичность сочетается с наличием обсценной лексики, на основе этого наблюдения было решено выделить ее из сообщений, что преследовало двойную цель:
- демаскировать скрытую грубость, когда оскорбления встраиваются в текст без пробелов
- усилить вес подобных слов

В исследование вошли следующие этапы:
- Проверка данных на готовность к обработке
- Выделение обсценной лексики из сообщений в отдельный признак
- Обработка основного текстового признака
    - токенизация с фильтрация неалфавитных символов
    - фильтрация по длине и количеству слов
    - удаление стоп-слов
    - стеммизация
- формирование нового признака из обработанного текстового признака и признака обсценных слов
- Создание конвейера автоматизированной обработки с
    - векторизатором на основе метрики tf-idf
    - ridge-классификатором
- кросс-валидация и подбор гиперпараметров векторизатора и классификатора вручную
- Замена ridge-классификатора на  несколько других вариантов поочередно
- Формулирование  выводов
- Финальная проверка на тестовой выборке

По результатам проведенных мероприятий была предложена схема на основе tf-idf-векторизатора  и ridge-классификатора, как наиболее оптимальная по временным и вычислительным затратам, при этом дающая необходимый уровень качества



In [1]:
import os
import re

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import RidgeClassifier, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

import nltk

nltk.download("wordnet")
nltk.download("stopwords")
from nltk.corpus import wordnet, stopwords
from nltk.stem import PorterStemmer

from tqdm import tqdm

tqdm.pandas()

from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True, nb_workers=8)


INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\monah\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\monah\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success">
Отлично, все нужные библиотеки импортированы в начале ноутбука.Это хорошая практика.

Также вижу, что ты "расчехлил" pandarell для ускорения обработки ))
</div>

In [2]:
pd.options.display.max_colwidth = None
pd.options.display.max_rows = None

In [3]:
def find_pattern(text, pattern):
    """Returns a string containing all words in the pattern"""
    import re
    return ' '.join(re.findall(pattern, text.lower()))


def get_max_length(text):
    """Returns the longest word in the text"""
    import numpy as np
    if len(text) == 0: return ''
    words = text.split()
    lengths = [len(w) for w in words]
    return words[np.argmax(lengths)]

def preprocess(text, max_word_length=32, max_words_number=500):
    import re
    from nltk.stem import PorterStemmer
    from nltk.corpus import stopwords
    ps = PorterStemmer()
    stop_words = stopwords.words('english')
    lst = re.sub(r'[^A-Za-z]', ' ', text).split()[:max_words_number]
    stems = [ps.stem(w) for w in lst if len(w) <= max_word_length and w not in stop_words]
    return ' '.join(stems)

## Загрузка и изучение данных

In [4]:
if os.path.exists('/datasets/toxic_comments.csv'):
    data = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)
elif os.path.exists('toxic_comments.csv'):
    data = pd.read_csv('toxic_comments.csv', index_col=0)
else:
    print('download failed')

In [5]:
data.head()

,text,toxic
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,"You, sir, are my hero. Any chance you remember what page that's on?",0


In [6]:
display(data[data.toxic == 1].sample(5, random_state=42))
display(data[data.toxic == 0].sample(5, random_state=42))

,text,toxic
92964,"Dude, get a fucking life you queer.. get a real paying job instead of e-wiki policing around, you 50 year old piece of shite... and your mom said to make your bed.",1
120398,The only nonsense I will approve is nonsense committed by 1)chaka zulu warriors. 2)anglican gay priests 3)cocksuck ers,1
47519,Thou shalt be ravished in thy buttocks \n\nIt's sure to be hurtin!,1
815,"Groin \n\nYou sad fuck. I'm going to restore my edit, and you are going to leave it, or else I wil knee you in the groin yourself, prick. 91.108.208.101",1
55660,"Cause of Death \n\nFuck exicution, put down Murder was the mutha fuckin' case, and Shawtnegger was tha murdera!",1


,text,toxic
145956,"Press \n June 2011: Today Show, NBC: Today's Money: Money-Maximizing Web Sites\n June 2011: MORE Magazine: Make Even More Money\n June 2011: Mint.com: Free Credit Scores (Seriously, They’re Really Free)\n March 2011: The Wall Street Journal: The Daily Start-up\n December 2010: Money Crashers: Credit Sesame Review – Free Credit and Debt Management Tool\n October 2010: Consumer Reports: Finovate Fall Day 1: Financial voyeurism and a free credit score \n September 2010: CNN Money: Today in Tech: News around the Web",0
41086,"Contested deletion \n\nThis article should not be speedy deleted as lacking sufficient context to identify its subject, because... there is an article on Wikipedia:\n\nhttp://nl.wikipedia.org/wiki/Dru_Yoga",0
127657,"""\n Your latest edits have goen even further towards a Christian POV of the article. For an article on the historicity of Jesus, see that article. This article is not about historicity at all, but you've just gone and made edits that make it seem like it is. And plus, you've dramatically changed what the sources are saying. \nYou see, what you guys don't seem to understand is that \n\n""""The term Historical Jesus refers to scholarly reconstructions of the life of Jesus of Nazareth,[3][4][5""""\n\nIs not the same as \n\n""""The term Historical Jesus refers to scholarly reconstructions of portraits of the life of Jesus of Nazareth.[3][4][5]""""\n\nTo go out and dramatically change what the sources are saying is probably a gross misrepresentation of those sources. """,0
5828,"""\nIt should not be deleted, but fixed. North Kosovo is de facto independent from the Albanian-dominated government in Pristina, while other enclaves south are not. speaks """,0
87827,"I'm back... \n\n...I haven't found the rusty knife yet, but I'm working on it. Now, I believe we have matters to discuss. Like wtf do you think you are playing at?!?! 217.41.238.156",0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [8]:
data.toxic.value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [9]:
data.text.duplicated().sum()

0

Данные загружены и осмотрены. Пропусков и явных дубликатов нет, Целевой признак сильно разбалансирован.
Изучим длину слов в сообщениях, поскольку наличие сверхдлинных слов может помешать дальнейшей обработке

In [10]:
data[ 'longest_word'] = data.text.parallel_apply(get_max_length)
data['max_word_length'] = data.longest_word.str.len()
data.sort_values(by='max_word_length', ascending=False).head(10)


ZeroDivisionError: float division by zero

А также длину сообщений и количество слов в них

In [ ]:
data['length'] = data.text.str.len()
data.length.describe()

In [ ]:
display(data[data.toxic == 1]['length'].describe())
display(data[data.toxic == 0]['length'].describe())

In [ ]:
data['n_words'] = data.text.apply(lambda x: len(x.split()))
data.n_words.describe()

Сверхдлинные слова чаще всего сочетаются с токсичностью, среднее количество слов в сообщениях - около семидесяти, но есть и гораздо более длинные сообщения.
Общая длина сообщений по количеству символов ограничена пятью тысячами.
Далее мы создадим паттерн с некоторыми грубыми лексемами, создадим признаки на его основе и изучим соответствие наличия грубости токсичности сообщения.

## Паттерн грубости

In [ ]:
rudeness_pattern = re.compile(r'fuck|fuk|bitch|cunt|basta|puke|vomit|boner|sod off|bugger|tosser| cock|nigger|nigga|hate you|piss off|shut up|suck|shit|damn|dick|twat|queer|faggot|jerk|buttocs|dumb|bloody')
data['rude'] = data.text.parallel_apply(find_pattern, pattern=rudeness_pattern)
data['rudeness'] = data.rude.progress_apply(lambda x: len(x) > 0)
print('Done')

In [ ]:
display(data[data.rudeness & (data.toxic == False)].head(10))
display(data[~data.rudeness & (data.toxic == True)].head(10))

In [ ]:
print(data[data.rudeness & data.toxic == 1].shape[0])
print(data[~data.rudeness & data.toxic == 1].shape[0])

В половине случаев токсичность соответствует наличию обсценной лексики, посмотрим, каково будет качество предсказаний по метрике f1, если прямолинейно  использовать наличие грубости в сообщениях, как показатель их токсичности.

In [ ]:
f1_score(data.toxic, data.rudeness.astype(int))

Приравнивание наличие грубости токсичности сообщения дает хорошее отправное значение для дальнейших мероприятий.

 Наконец посмотрим, как соответствует количество грубых сообщений максимальной длине слов в них.

In [ ]:
# Признак грубости и длина слов  
step = 10
for length in np.arange(data.max_word_length.min(), data.max_word_length.max(), step):
    print(f'{length} - {length + step}:', data[(data.max_word_length >= length) & (data.max_word_length < length + step)]['rudeness'].sum())

Действительно, хотя подавляющее большинство грубости не соответствует наличию сверхдлинных слов, но таковые встречаются и могут маскировать грубые выражения.
Таким образом, на этапе изучения были сделаны следующие выводы и предположения:
- Данные, в целом, готовы для дальнейшей работы, но
    - необходимо учесть наличие очень длинных слов, не являющихся URL-адресами, что может значительно увеличить время  обработки текста, вызвать исключения и снизить в известной степени качество предсказаний за счет маскировки обсценной лексики
    - необходимо также учесть наличие некоторого количества длинных сообщений
- Выделение и дальнейшее использование признака грубости гипотетически может улучшить качество работы модели, поскольку в половине случаев этот признак соответствует токсичности

Ниже мы подготовим основной текстовый признак, наборы данных и перейдем к работе с векторизацией и моделями

## Предобработка

В ходе серии экспериментов была подобрана оптимальная схема предобработки текстового признака, инкапсулированная в функции preprocess().
Эта схема включает в себя:
- очистку от неалфавитных символов
- фильтрацию по длине слов и количеству слов в сообщении (более длинные слова выбрасываются, более длинные сообщения обрезаются)
- фильтрацию стоп-слов
- стеммизацию
- обратную сборку текста

Эксперименты показали, что стеммизация дает лучшие результаты, чем лемматизация, отсутствие приведения к одному регистру никак не проявляется,а фильтрация  значительно улучшает скорость работы.
Фильтрация же URL-адресов негативно сказывается на качестве.

Далее мы обогатим полученный признак за счет признака грубости, разделим выборку на обучающую и тестовую в соотношении 3 : 1с учетом дисбаланса.

In [ ]:
data['stems'] = data.text.parallel_apply(preprocess)


In [ ]:
data['stems_rude'] = data.stems+ ' ' + data.rude



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.stems_rude, data.toxic, test_size=.1, stratify=data.toxic, random_state=42)

##  Создание конвейера автоматизированной обработки

Конвейер состоит из TF-IDF-векторизатора и Ridge-классификатора, гиперпараметры которых подбирались вручную на небольшом сэмпле данных.
Вначале посмотрим, какой результат получается при использовании подготовленного и обогащенного текстового признака, затем, чтобы убедиться в целессообразности примененной в отношении табуированной лексики манипуляции проведем проверку с использованием чистого подготовленного признака.

In [ ]:
vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    lowercase=False,
    max_features=None,
    max_df=0.3,
    sublinear_tf=True,
    smooth_idf=False,
)
classifier = RidgeClassifier(alpha=1.2, random_state=42, class_weight="balanced")
pipe = Pipeline(steps=[("vectorizer", vectorizer), ("classifier", classifier)])


In [ ]:
cross_validate(pipe, X_train, y_train, cv=3,
               scoring='f1', n_jobs=-1)['test_score'].mean()

In [ ]:
X_train_stems, _, y_train_stems, _ = train_test_split(data.stems, data.toxic, random_state=42, stratify=data.toxic)
cross_validate(pipe, X_train_stems, y_train_stems, scoring='f1', n_jobs=-1, cv=3)['test_score'].mean()

Примененная техника дает прирост качества в пару процентов, основная же эффективность достигается за счет настройки векторизатора. Гиперпараметры классификатора, за исключением alpha, остались на заводских значениях.

## Апробация других моделей

Здесь мы попробуем применить модели на основе машины опорных векторов, градиентного спуска, к-ближайших соседей и случайного леса. Заранее скажем, что все они, особенно первая, гораздо более требовательны к временным ресурсам и дают (в особенности третья) гораздо худший результат.

In [ ]:
pipe.set_params(classifier=SVC(random_state=42))
cross_validate(pipe, X_train, y_train, scoring='f1', n_jobs=-1, cv=3)['test_score'].mean()

In [ ]:
pipe.set_params(classifier=SGDClassifier(random_state=42, n_jobs=-1, early_stopping=True))
cross_validate(pipe, X_train, y_train, scoring='f1', n_jobs=-1, cv=3)['test_score'].mean()

In [ ]:
pipe.set_params(classifier=KNeighborsClassifier(n_jobs=-1))
cross_validate(pipe, X_train, y_train, scoring='f1', cv=3, n_jobs=-1)['test_score'].mean()

In [ ]:
pipe.set_params(classifier=RandomForestClassifier(random_state=42, n_jobs=-1))
cross_validate(pipe, X_train, y_train, scoring='f1', n_jobs=-1, cv=3)['test_score'].mean()

Таким образом, лучшие показатели, как по требуемым ресурсам, так и по качеству, у модели на основе гребневой регрессии. Именно ее в сочетании с TF-IDF-векторизатором мы и рекомендуем для дальнейшего использования.
Чтобы убедиться в адекватности ее работы, проверим предсказания на тестовой выборке, обучив модель на тренировочной.

## Финальное тестирование

In [ ]:
pipe.set_params(classifier = RidgeClassifier(alpha=1.2, random_state=42, class_weight='balanced'))
pipe.fit(X_train, y_train)
predictions = pipe.predict(X_test)
f1_score(y_test, predictions)

Даже лучше, чем средний показатель при кросс-валидации на обучающей выборке, что свидетельствует о полном отсутствии переобучения.
Взглянем на более подробный отчет.

In [ ]:
pd.DataFrame(classification_report(y_test, predictions, output_dict=True))

Есть некоторые сложности в правильном предсказании метки 1, что вполне ожидаемо.

## Заключение

В ходе настоящей работы были изучены и обработаны данные в виде набора текстовых сообщений с целью подобрать максимально эффективный механизм определения их токсичности.
Для увеличения эффективности предсказания была применена техника выделения обсценной лексики для усиления сообветствующего текста.
Текстовый признак был обработан с помощью фильтрации по характеру символов, длине и количеству слов и  стеммизации.
Использовалась TF-IDF-векторизация, был проведен ручной поиск ее гиперпараметров,  был проведен поиск среди нескольких моделей машинного обубчения, в результате которого выбор был сделан в пользу классификации на основе гребневой регрессии.
        Было проведено финальное тестирование, подтвердившее работоспособность выбранной схемы.Цели и задачи работы полностью реализованы, исследование успешно закончено.